In [9]:
import pandas as pd
import json
import re

df = pd.read_pickle('mistral_results.pkl')

def extract_value(json_string, key):
    pattern = f'"{key}": (.*?)(?:,|\n)'
    match = re.search(pattern, json_string)
    return match.group(1).strip() if match else None

keys = [
    "relevant_facts",
    "contained_explicit_hate_speech_vocabular",
    "argument_for_hate_speech",
    "argument_against_hate_speech",
    "likelihood_of_presence_of_implicit_hate_speech",
    "likelihood_of_presence_of_explicit_hate_speech",
    "likelihood_of_ad_hominem_attack",
    "likelihood_of_minority_attack",
    "likelyhood_of_takedown_on_social_media",
    "hate_speech_probability"
]

for key in keys:
    df[key] = df['answer'].apply(lambda x: extract_value(x, key))

In [10]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values for each column:")
print(missing_values)

# Check for duplicated rows
duplicated_rows = df.duplicated().sum()
print(f"\nNumber of duplicated rows: {duplicated_rows}")

# Check the data types of each column
data_types = df.dtypes
print("\nData types for each column:")
print(data_types)

Missing values for each column:
model                                              0
time                                               0
promt_hash                                         0
prompt                                             0
answer                                             0
truth                                              0
relevant_facts                                     3
contained_explicit_hate_speech_vocabular          10
argument_for_hate_speech                          24
argument_against_hate_speech                      53
likelihood_of_presence_of_implicit_hate_speech     9
likelihood_of_presence_of_explicit_hate_speech    18
likelihood_of_ad_hominem_attack                    5
likelihood_of_minority_attack                      3
likelyhood_of_takedown_on_social_media            15
hate_speech_probability                            4
dtype: int64

Number of duplicated rows: 0

Data types for each column:
model                                           

In [11]:
df

,model,time,promt_hash,prompt,answer,truth,relevant_facts,contained_explicit_hate_speech_vocabular,argument_for_hate_speech,argument_against_hate_speech,likelihood_of_presence_of_implicit_hate_speech,likelihood_of_presence_of_explicit_hate_speech,likelihood_of_ad_hominem_attack,likelihood_of_minority_attack,likelyhood_of_takedown_on_social_media,hate_speech_probability
0,mistral,2024-01-08 15:30:47.202459,-3450054670443982230,I 've always considered teaching as one of the...,"{ \n ""relevant_facts"": [\n ""The speaker ex...",True,[,[,"""The speaker uses derogatory language to descr...","""The speaker is merely expressing a personal p...",0.32,0.63,0.11,0,0.54,0.63
1,mistral,2024-01-08 15:30:50.432009,-3450054670443982230,And the sad thing is the white students at tho...,"{ \n ""relevant_facts"": [\n ""The sentence r...",True,[,[],"""The sentence implies a negative generalizatio...","""The sentence does not include any explicitly ...",0.36,0.00,0.12,0.63,0.51,0.36
2,mistral,2024-01-08 15:30:52.942114,-3450054670443982230,This whole situation starts to become really b...,"{ \n ""relevant_facts"": [],\n ""contained_expl...",False,[],[],"""The sentence does not contain any explicit ha...","""The sentence expresses a subjective feeling o...",0,0,0,0,0,0
3,mistral,2024-01-08 15:30:55.288015,-3450054670443982230,That's all I needed to hear .,"{ \n ""relevant_facts"": [],\n ""contained_expl...",False,[],[],"""""",""" Without the provided sentence",0,0,0,0,0,0
4,mistral,2024-01-08 15:30:58.367100,-3450054670443982230,Excellent Article !! * * * * * Why Were n't Th...,"{ \n ""relevant_facts"": [\n ""The sentence i...",False,[,[],"""The title of the article and the question its...","""The lack of explicit hate speech vocabulary a...",0.01,0.0,0.01,0.02,0.05,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,mistral,2024-01-08 15:55:52.206169,-3450054670443982230,I have one word for you ....... MOVE . We chos...,"{ \n ""relevant_facts"": [""statement expresses ...",False,"[""statement expresses desire for all-white sch...","[""white""]","""Statement expresses preference for children t...","""The statement is not directly calling for har...",0.88,None,None,0.85,0.9,0.77
474,mistral,2024-01-08 15:55:54.653929,-3450054670443982230,There are only eight people living within a mi...,"{ \n ""relevant_facts"": [""The sentence describ...",False,"[""The sentence describes the number of people ...",[],"""The sentence does not contain any hate speech.""","""The sentence is a factual statement about the...",0.0,0.0,0.0,0.0,0.0,0.0
475,mistral,2024-01-08 15:55:57.710958,-3450054670443982230,you just say blacks hate the cold. the only re...,"```json\n{ \n ""relevant_facts"": [""The sentenc...",True,"[""The sentence contains stereotypes about Blac...","[""Blacks""","""The sentence makes a derogatory statement abo...","""There is no argument against hate speech in t...",0,1,0,1,1,1
476,mistral,2024-01-08 15:56:00.423109,-3450054670443982230,"And all those holes below 40,000 feet are fill...","{ \n ""relevant_facts"": [\n ""The sentence d...",False,[,[],"""There is no apparent hate speech in this sent...","""None directly related to this sentence.""",0.01,0.0,0.0,0.0,0.01,0.0


In [13]:
from sklearn.metrics import roc_auc_score, log_loss, brier_score_loss

# Assuming y_true is your array of true labels and y_pred is your array of predicted probabilities
y_true = merged_df['label']
y_pred = merged_df['prediction']

roc_auc = roc_auc_score(y_true, y_pred)
logloss = log_loss(y_true, y_pred)
brier_score = brier_score_loss(y_true, y_pred)

print(f"ROC AUC Score: {roc_auc}")
print(f"Log Loss: {logloss}")
print(f"Brier Score: {brier_score}")

NameError: name 'merged_df' is not defined

In [ ]:
processed_data_df = pd.read_pickle('processed_data.pkl')

merged_df = pd.merge(data_df, processed_data_df, on='text')